In [1]:
!pip install pydrive

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
import tensorflow as tf
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
zip_file = drive.CreateFile({'id': '1C5FOGXLAAhvdaOuTWukGysyuj1NjIpVy'})
zip_file.GetContentFile('complete_data_reduced.zip')
validation_file = drive.CreateFile({'id':'1Xiy766wWCAo8gcNSyEFzwn2xaUHtxXqx'})
validation_file.GetContentFile('validation_reduced.zip')

In [0]:
# !unzip 'complete_data_reduced.zip'
# !unzip 'validation_reduced.zip'

In [5]:
from keras.models import Sequential
from keras.utils.np_utils import to_categorical  
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import Convolution2D,SeparableConv2D,Conv2D
from keras.optimizers import Adam, RMSprop
from keras.layers.pooling import MaxPooling2D,AveragePooling2D,GlobalAveragePooling2D
from keras.layers import BatchNormalization,Activation,Input
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob,string

## file paths training data
angry_path = 'combined_data_reduced/angry/*'
closed_path = 'combined_data_reduced/closed/*'
disgust_path = 'combined_data_reduced/disgusted/*'
fear_path = 'combined_data_reduced/fearful/*'
happy_path = 'combined_data_reduced/happy/*'
neutral_path = 'combined_data_reduced/neutral/*'
sad_path = 'combined_data_reduced/sad/*'
surprised_path = 'combined_data_reduced/surprised/*'

## file path validation
angry_path_val = 'validation_reduced/angry/*'
closed_path_val = 'validation_reduced/closed/*'
disgust_path_val = 'validation_reduced/disgusted/*'
fear_path_val = 'validation_reduced/fearful/*'
happy_path_val = 'validation_reduced/happy/*'
neutral_path_val = 'validation_reduced/neutral/*'
sad_path_val = 'validation_reduced/sad/*'
surprised_path_val = 'validation_reduced/surprised/*'


#list files training
angry_files=glob.glob(angry_path)
closed_files = glob.glob(closed_path)
disgust_files = glob.glob(disgust_path)
fear_files = glob.glob(fear_path)
happy_files = glob.glob(happy_path)
neutral_files = glob.glob(neutral_path)
sad_files = glob.glob(sad_path)
surprised_path = glob.glob(surprised_path)

#list files validation
angry_files_val=glob.glob(angry_path_val)
closed_files_val = glob.glob(closed_path_val)
disgust_files_val = glob.glob(disgust_path_val)
fear_files_val = glob.glob(fear_path_val)
happy_files_val = glob.glob(happy_path_val)
neutral_files_val = glob.glob(neutral_path_val)
sad_files_val = glob.glob(sad_path_val)
surprised_path_val = glob.glob(surprised_path_val)

In [0]:
batch_size = 32
num_epoch = 30

training_directory='combined_data_reduced'
validation_directory = 'validation_reduced'

In [8]:
train_datagen = ImageDataGenerator(rescale=1./255,
                        featurewise_center=False,
                        featurewise_std_normalization=False,
                        rotation_range=10,
                        width_shift_range=0.1,
                        height_shift_range=0.1,
                        zoom_range=.1,
                        horizontal_flip=True
                                  )
val_datagen = ImageDataGenerator(rescale=1./255,
                        featurewise_center=False,
                        featurewise_std_normalization=False,
                        rotation_range=10,
                        width_shift_range=0.1,
                        height_shift_range=0.1,
                        zoom_range=.1,
                        horizontal_flip=True
                                )

train_generator = train_datagen.flow_from_directory(
        training_directory,
        target_size=(48,48),
        batch_size=batch_size,
        color_mode="grayscale",
        class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
        validation_directory,
        target_size=(48,48),
        batch_size=batch_size,
        color_mode="grayscale",
        class_mode='categorical')


Found 31527 images belonging to 8 classes.
Found 7591 images belonging to 8 classes.


In [0]:
def create_model():
    
    model = Sequential()

    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
#     model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(AveragePooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, kernel_size=(2, 2), activation='relu'))
    model.add(AveragePooling2D(pool_size=(2, 2)))
    model.add(Conv2D(256, kernel_size=(1, 1), activation='relu'))
    model.add(Conv2D(512, kernel_size=(1, 1), activation='relu'))
    model.add(AveragePooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(8, activation='softmax'))
 
    return model

In [10]:
model = create_model()
#model = create_model2()
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 46, 46, 32)        320       
_________________________________________________________________
average_pooling2d_1 (Average (None, 23, 23, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 22, 22, 128)       16512     
_________________________________________________________________
average_pooling2d_2 (Average (None, 11, 11, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)    

In [0]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
  json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")

In [12]:
!mkdir 'trained_models'

mkdir: cannot create directory ‘trained_models’: File exists


In [0]:
from keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
patience =12

#log_file_path = base_path + dataset_name + '_emotion_training.log'
#csv_logger = CSVLogger(log_file_path, append=False)
early_stop = EarlyStopping('val_loss', patience=patience)
reduce_lr = ReduceLROnPlateau('val_loss', factor=0.1,
                                  patience=int(patience/4), verbose=1)
trained_models_path = 'trained_models3'
model_names = trained_models_path + '.{epoch:02d}.hdf5'
model_checkpoint = ModelCheckpoint(model_names, 'val_loss', verbose=1,
                                                    save_best_only=True)
callbacks = [model_checkpoint, early_stop, reduce_lr]

In [14]:
model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.0001),metrics=['accuracy'])

model_info = model.fit_generator(
            train_generator,
            
            steps_per_epoch=950, #len(dataset)/batch_size
            epochs=num_epoch,
            callbacks=callbacks,
            validation_data=validation_generator,
            validation_steps=200,
            workers=4,
            shuffle=True)

Instructions for updating:
Use tf.cast instead.
Epoch 1/30
950/950 [==============================] - 83s 87ms/step - loss: 1.9025 - acc: 0.2365 - val_loss: 1.8719 - val_acc: 0.2594

Epoch 00001: val_loss improved from inf to 1.87186, saving model to trained_models3.01.hdf5
Epoch 2/30
950/950 [==============================] - 79s 83ms/step - loss: 1.8497 - acc: 0.2732 - val_loss: 1.8330 - val_acc: 0.2919

Epoch 00002: val_loss improved from 1.87186 to 1.83304, saving model to trained_models3.02.hdf5
Epoch 3/30
950/950 [==============================] - 78s 82ms/step - loss: 1.8154 - acc: 0.2930 - val_loss: 1.7961 - val_acc: 0.2993

Epoch 00003: val_loss improved from 1.83304 to 1.79614, saving model to trained_models3.03.hdf5
Epoch 4/30
950/950 [==============================] - 77s 81ms/step - loss: 1.7828 - acc: 0.3116 - val_loss: 1.7719 - val_acc: 0.3053

Epoch 00004: val_loss improved from 1.79614 to 1.77190, saving model to trained_models3.04.hdf5
Epoch 5/30
950/950 [============